In [1]:
import syft as sy
import torch as t
import numpy as np
import pandas as pd

## Launch a Duet Sever

In [2]:
duet = sy.launch_duet(loopback = True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...

♫♫♫ > CONNECTED!



## Read Data and Preprocess

In [3]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_1.csv")  

In [4]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [5]:
# Trnasform the input to tensor
X_train_tensor = t.FloatTensor(X_train)
y_train_tensor = t.tensor(y_train,dtype=t.long)
X_test_tensor = t.FloatTensor(X_test)
y_test_tensor = t.tensor(y_test,dtype=t.long)

## Send data to Duet server

In [9]:
X_train_tensor.tag("X_Train_Owner1")
X_train_tensor.describe("Dataset of 400 samples, 7 feature")
X_train_ptr = X_train_tensor.send(duet,pointable = True)

y_train_tensor.tag("y_Train_Owner1")
y_train_tensor.describe("Dataset of 400 samples, 1 feature")
y_train_ptr = y_train_tensor.send(duet,pointable = True)

X_test_tensor.tag("X_Test_Owner1")
X_test_tensor.describe("Dataset of 400 samples, 7 feature")
X_test_ptr = X_test_tensor.send(duet,pointable = True)

y_test_tensor.tag("y_Test_Owner1")
y_test_tensor.describe("Dataset of 400 samples, 1 feature")
y_test_ptr = y_test_tensor.send(duet,pointable = True)


## Create a request handler

In [7]:
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)

In [10]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 27034ee5bb12433b9a5ad4f8d292f7a2>,[X_Train_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: 6a775e86c0534b03a3acebdead61c60f>,[y_Train_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: 65072e4612e44795b753a24d66cd0d66>,[X_Test_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 3f08d213eece4aaa8b5a2d281095c2fc>,[y_Test_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


[2022-06-21T21:55:55.319286-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 936fbd73db604971b22d42815cf065e1>: To evaluate training progress
Value: <Storable: torch.Size([400, 7])>
[2022-06-21T21:58:29.044806-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: fd10e723596a4d03bbbaeefff0360f4f>: To evaluate training progress
Value: <Storable: torch.Size([400, 7])>
[2022-06-21T21:59:26.808917-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 5ea5512d05d74894a17dbe25317bb3a0>: To evaluate training progress
Value: <Storable: torch.Size([400, 7])>
[2022-06-21T22:00:00.323688-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 76832d0689424dfc8741ce196ecca3e9>: To evaluate training progress
Value: <Storable: torch.Size([400, 7])>
[2022-06-21T22:05:19.018339-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 8e6b5efa074f420f99af5ee9a0ca9b0b>: To evaluate training progress
Value: <Storable: 0.6723041534423828>
[2022-06-21T22:05:19.187573-0400][CRITICAL][logger]][1304

[2022-06-21T22:05:26.301929-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 38055b1eae7247598ac9036a2f26c24a>: To evaluate training progress
Value: <Storable: 0.6453841328620911>
[2022-06-21T22:05:26.457933-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 2a6b0bc1d9b944f7a5844714df809b2b>: To evaluate training progress
Value: <Storable: 0.6408918499946594>
[2022-06-21T22:05:26.611829-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: bfb27677713b489f92cca0010b33b4c7>: To evaluate training progress
Value: <Storable: 0.6407157778739929>
[2022-06-21T22:05:26.779135-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: e57e99d2da9a4f20b4ca80a3e3be921e>: To evaluate training progress
Value: <Storable: 0.640465497970581>
[2022-06-21T22:05:26.928095-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 2858feed52b54234948eff28bd8c939c>: To evaluate training progress
Value: <Storable: 0.6408908367156982>
[2022-06-21T22:05:27.092101-0400][CRITICAL][logger]][13044] > HAND

[2022-06-21T22:05:33.415532-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 2dc8f8fa51db480a8460fb58358d8f3d>: To evaluate training progress
Value: <Storable: 0.6140607595443726>
[2022-06-21T22:05:33.577086-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: da423d96f9004441a22433c72d60ede1>: To evaluate training progress
Value: <Storable: 0.616361677646637>
[2022-06-21T22:05:33.737086-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: a08f1aeff18c46fdbdaaf81fdfb3fddc>: To evaluate training progress
Value: <Storable: 0.6141966581344604>
[2022-06-21T22:05:33.898086-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 2c0f0a6a58a441dbbb8d06266ddf8243>: To evaluate training progress
Value: <Storable: 0.6149453520774841>
[2022-06-21T22:05:34.067081-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 512d356be5f24714b1a8b13fa763e9f6>: To evaluate training progress
Value: <Storable: 0.614773154258728>
[2022-06-21T22:05:34.230101-0400][CRITICAL][logger]][13044] > HANDL

[2022-06-21T22:05:42.328386-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 1a0d0769413e439ba2dcafe8250845d9>: To evaluate training progress
Value: <Storable: 0.5921820998191833>
[2022-06-21T22:05:42.485434-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 0e2708c8c62e4591ace2bfcf51970121>: To evaluate training progress
Value: <Storable: 0.5911657214164734>
[2022-06-21T22:05:42.653932-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: c641acaa91334f6ca54a529a67fcebba>: To evaluate training progress
Value: <Storable: 0.5932781100273132>
[2022-06-21T22:05:42.811685-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 0cbc134204474b969afef283d20314a3>: To evaluate training progress
Value: <Storable: 0.5907735228538513>
[2022-06-21T22:05:42.969888-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 84713869b9ff4424bd037b72e1d3c87f>: To evaluate training progress
Value: <Storable: 0.5918951630592346>
[2022-06-21T22:05:43.124114-0400][CRITICAL][logger]][13044] > HAN

...

[2022-06-21T22:47:56.991957-0400][CRITICAL][logger]][13044] sum() received an invalid combination of arguments - got (axis=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: axis, out
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)

[2022-06-21T22:47:56.992958-0400][CRITICAL][logger]][13044] sum() received an invalid combination of arguments - got (axis=NoneType, out=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: axis, out
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)

Exception in callback AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...pe dtype)\n")>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>.callback(

[2022-06-21T22:48:43.516065-0400][CRITICAL][logger]][13044] sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)

[2022-06-21T22:48:43.516065-0400][CRITICAL][logger]][13044] sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)

Exception in callback AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...pe dtype)\n")>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>.callback(

[2022-06-21T22:49:05.823752-0400][CRITICAL][logger]][13044] <class 'syft.core.store.store_memory.MemoryStore'> __getitem__ error <UID: a5e2c9437a364147b33c2866690fb1c8> <UID: a5e2c9437a364147b33c2866690fb1c8>
[2022-06-21T22:49:05.823922-0400][CRITICAL][logger]][13044] <UID: a5e2c9437a364147b33c2866690fb1c8>
[2022-06-21T22:49:05.823922-0400][CRITICAL][logger]][13044] <UID: a5e2c9437a364147b33c2866690fb1c8>
[2022-06-21T22:49:05.823922-0400][CRITICAL][logger]][13044] <UID: a5e2c9437a364147b33c2866690fb1c8>
Exception in callback AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...866690fb1c8>)>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...866690fb1c8>)>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65>
Traceback (most recent call last):
  File "D:\anaconda\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "D:\anaconda\lib\site-packages\pye

[2022-06-21T23:00:48.141802-0400][CRITICAL][logger]][13044] <class 'syft.core.store.store_memory.MemoryStore'> __getitem__ error <UID: 785b94c54fee471d8bf6179f7072137a> <UID: 785b94c54fee471d8bf6179f7072137a>
[2022-06-21T23:00:48.141802-0400][CRITICAL][logger]][13044] <UID: 785b94c54fee471d8bf6179f7072137a>
[2022-06-21T23:00:48.141802-0400][CRITICAL][logger]][13044] <UID: 785b94c54fee471d8bf6179f7072137a>
[2022-06-21T23:00:48.141802-0400][CRITICAL][logger]][13044] <UID: 785b94c54fee471d8bf6179f7072137a>
Exception in callback AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...79f7072137a>)>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65
handle: <Handle AsyncIOEventEmitter._emit_run.<locals>.callback(<Task finishe...79f7072137a>)>) at D:\anaconda\lib\site-packages\pyee\asyncio.py:65>
Traceback (most recent call last):
  File "D:\anaconda\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "D:\anaconda\lib\site-packages\pye

[2022-06-21T23:05:59.733949-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: c7413cf8bc0b416fac907ff8b7603e91>:
Value: <Storable: 'in_features=7, out_features=10, bias=True'>
[2022-06-21T23:05:59.881515-0400][CRITICAL][logger]][13044] > HANDLER Request <UID: 7be547ccc5e74cd287735b568a9c8686>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.0779, 0.0569, 0.2021, -0.1684, -0.0360, 0.2680, -0.1703],    [-0.2216, -0.1781, 0.1602, 0.0738, -0.0785, 0.3156, 0.2587],    [ 0.1028, -0.0767, -0.0728, -0.2424, 0.0311, 0.3255, -0.2238],    [ 0.2700, -0.2687, 0.2896, 0.1487, 0.0590, -0.2297, 0.0506],    [ 0.0889, -0.0486, 0.0543, 0.0124, 0.5594, 0.1218, 0.0915],    [-0.2139, 0.1943, -0.0509, 0.1414, 0.2502, -0.2040, 0.1275],    [ 0.1308, 0.0856, 0.3098, -0.1076, 0.2109, 0.0107, -0.1479],    [ 0.1113, -0.3909, -0.0487, 0.1316, 0.0051, -0.0901, 0.3607],    [-0.1767, -0.1866, -0.1866, -0.3488, -0.0737, 0.1476, 0.0562],    [ 0.3401, -0.3482, 0.0943, -0.0443, -0.2453, -0.2647, 0.0708]])), ('

## Build a local Classifier

In [161]:
class Net(t.nn.Module):
    def __init__(self,input_channels,output_channels):
        super(Net,self).__init__()
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = t.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = t.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = t.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = t.nn.ReLU()
        self.dropout = t.nn.Dropout(p=0.1)
        self.batchnorm1 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = t.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = t.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = t.argmax(output,1)
        return prediction
        


In [149]:
def classifier_train(epochs, model, optimizer, X,y,criterion):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred = model.forward(X)
        #Compute Cross entropy loss
        loss = criterion(y_pred,y)
        #Add loss to the list
        losses.append(loss.item())
        
        #Print loss
        if i%500==0:
            print("Epoch:",i," Loss:",loss.item())
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses


## Local Training

In [162]:
# Define the input and output
input_channels = X_train_tensor.shape[1]
output_channels = 2

#Initialize the model        
model = Net(input_channels,output_channels)
#Define loss criterion
criterion = t.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = t.optim.Adam(model.parameters(), lr=0.001)
#Number of epochs
epochs = 5000

losses = classifier_train(
    epochs, model, optimizer,X_train_tensor,y_train_tensor,criterion
)

Epoch: 0  Loss: 0.6961112022399902
Epoch: 500  Loss: 0.4703548550605774
Epoch: 1000  Loss: 0.4529033899307251
Epoch: 1500  Loss: 0.4467262923717499
Epoch: 2000  Loss: 0.434055894613266
Epoch: 2500  Loss: 0.4334057569503784
Epoch: 3000  Loss: 0.4298244118690491
Epoch: 3500  Loss: 0.4297804534435272
Epoch: 4000  Loss: 0.4262561500072479
Epoch: 4500  Loss: 0.4146350026130676


## Accuracy on test set

In [165]:
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [168]:
accuracy(model,X_test_tensor,y_test_tensor)

0.73
